## CS431/631 Big Data Infrastructure
### Winter 2018 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _replace this with your name_
* **ID:** _replace this with your UW student ID number_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [16]:
# Add the home directory to the system path for the interpreter to search from

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [27]:
# a.Number of distinct tokens

# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from Assignment_0.top50 import count_tokens

token_dic = count_tokens(filename='../Shakespeare.txt')
distinct_tokens_num = len(token_dic)
#print(distinct_tokens_num)

# The number of distinct tokens are 25975

# The number of distinct pairs is equal to the permuation of 2 items from distinct tokens
num_pairs = 25975*25974 #674674650

---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [30]:
class Query:
    def __init__(self):
        self._all_tokens = []
        self._spec_token = []
        self._text_to_tokens()

    def _text_to_tokens(self, filename="../Shakespeare.txt"):
        with open(filename) as f:
            for line in f:
                self._all_tokens.append(simple_tokenize(line))

    def _specific_token(self, token):
        self._spec_token = []
        for tokens in self._all_tokens:
            if token in tokens:
                self._spec_token.append(tokens)

    def _number_occur(self, token2):
        co_count = 0
        for tokens in self._spec_token:
            if token2 in tokens:
                co_count += 1
        return co_count

    def _two_token_query(self, token1, token2):
        nx = token_dic[token1]
        ny = token_dic[token2]
        nxy = self._number_occur(token2)
        total_lines = len(self._all_tokens)
        px = nx / total_lines
        py = ny / total_lines
        pxy = nxy / total_lines
        pmi = log(pxy / px * py)

        return pmi

    def consolidated_query(self, *q_tokens, threshold=None):
        n = 5
        q_tokens = q_tokens[0]

        if len(q_tokens) < 2:
            if threshold is None:
                raise Exception('Please provide a threshold for one-token query')

            else:
                token1 = q_tokens[0]
                pmi_list = []
                self._specific_token(token1)

                for token2 in token_dic:
                    if token2 == token1:
                        continue

                    n_co_occur = self._number_occur(token2)

                    if n_co_occur >= threshold:
                        pmi = self._two_token_query(token1, token2)
                        pmi_list.append((token1, token2, n_co_occur, pmi))

                pmi_list.sort(key=lambda x: x[3])
                pmi_n = pmi_list[:n]

        else:
            token1 = q_tokens[0]
            token2 = q_tokens[1]
            self._specific_token(token1)
            n_co_occur = self._number_occur(token2)
            pmi = self._two_token_query(token1, token2)
            pmi_n = [(token1, token2, n_co_occur, pmi)]

        return pmi_n


def main():

    query = Query()
    while True:
        q = input("Input 1 or 2 space-separated tokens (return to quit): ")
        if len(q) == 0:
            break
        q_tokens = simple_tokenize(q)
        if len(q_tokens) == 1:
            threshold = 0
            while threshold <= 0:
                try:
                    threshold = int(input("Input a positive integer frequency threshold: "))
                except ValueError:
                    print("Threshold must be a positive integer!")
                    continue

            print("  n({0}) = {1}".format(q_tokens[0], token_dic[q_tokens[0]]))
            print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0], threshold))
            lst = query.consolidated_query(q_tokens, threshold=threshold)

            for data in lst:
                token1, token2, n_co_occur, pmi = data
                print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(token1, token2, n_co_occur, pmi))

        elif len(q_tokens) == 2:
            lst = query.consolidated_query(q_tokens)
            for data in lst:
                token1, token2, n_co_occur, pmi = data
                print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(token1, token2, n_co_occur, pmi))

        else:
            print("Input must consist of 1 or 2 space-separated tokens!")


if __name__ == '__main__':
    main()

Input 1 or 2 space-separated tokens (return to quit): burn
Input a positive integer frequency threshold: 5
  n(burn) = 78
  high PMI tokens with respect to burn (threshold: 5):
    n(burn,fire) = 10,  PMI(burn,fire) = -8.18615081794896
    n(burn,doth) = 5,  PMI(burn,doth) = -7.616914432026555
    n(burn,up) = 5,  PMI(burn,up) = -7.467028150985755
    n(burn,them) = 5,  PMI(burn,them) = -6.871942183602024
    n(burn,out) = 7,  PMI(burn,out) = -6.871437260592054
Input 1 or 2 space-separated tokens (return to quit): out
Input a positive integer frequency threshold: 4
  n(out) = 1415
  high PMI tokens with respect to out (threshold: 4):
    n(out,eke) = 4,  PMI(out,eke) = -15.281528748998515
    n(out,knock'd) = 4,  PMI(out,knock'd) = -15.281528748998515
    n(out,drives) = 5,  PMI(out,drives) = -14.721912961063092
    n(out,inquire) = 5,  PMI(out,inquire) = -14.65292008957614
    n(out,stretch) = 4,  PMI(out,stretch) = -14.639674862826121
Input 1 or 2 space-separated tokens (return to qu

---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*Replace this with your answer to Question 3.*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.